In [4]:
import pandas as pd
import iris
import sys

sys.path.insert(1, '../')
sys.path.insert(1, '../Finding_AMAX_Events')
from Identify_Events_Functions import *
from Prepare_Data_Functions import *

yrs_range='2060_2081'
em='bb189'
home_dir = '/nfs/a319/gy17m2a/PhD/'

if yrs_range == '2060_2081':
    start_yr=2065
else:
    start_yr=2006

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv(home_dir + 'datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Read in a sample cube for finding the location of gauge in grid
sample_cube = iris.load(home_dir + f'datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{start_yr}01.nc')[0][1,:,:]

gauge_num=0
print(gauge_num)
Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

general_filename = home_dir + f'datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr*'
# Year isnt actually doing anything here 
cubes = load_files_to_cubelist(2061, general_filename)
print("Loaded data")
# Assuming your CubeList is called `cube_list`
filtered_cubes = iris.cube.CubeList()

# Define the coordinates to filter by
y_idx = idx_2d[0]
x_idx = idx_2d[1]

# Loop through each cube in the CubeList and filter
for cube in cubes:
    # Extract the data at the specified y_idx and x_idx
    filtered_cube = cube[:, y_idx, x_idx]
    
    # Append the filtered cube to the new CubeList
    filtered_cubes.append(filtered_cube)
    
print("Filtered to location")

# Join them into one (with error handling to deal with times which are wrong)
try:
    full_timeslice_cube = filtered_cubes.concatenate_cube()
    print("Concatenation successful!")
except Exception as e:
    print(f"Initial concatenation failed: {str(e)}")
    
    # If initial concatenation fails, remove problematic cubes and try again
    try:
        print(cube)
        full_timeslice_cube = remove_problematic_cubes(filtered_cubes)
        print("Concatenation successful after removing problematic cubes!")
    except RuntimeError as e:
        print(f"Concatenation failed after removing problematic cubes: {str(e)}")   
        
# print(f"Joined for {gauge_num}")       
# if not os.path.exists(home_dir + f'datadir/Gauge_Timeslices/2002_2020/{em}/'):
#     os.makedirs(home_dir + f'datadir/Gauge_Timeslices/2002_2020/{em}/')
# iris.save(full_timeslice_cube, home_dir + f'datadir/Gauge_Timeslices/2002_2020/{em}/gauge{gauge_num}_farFuture.nc')
# print("Saved")

0
Loaded data
Filtered to location
Initial concatenation failed: failed to concatenate into a single cube.
  Scalar coordinates metadata differ: forecast_reference_time != forecast_reference_time
stratiform_rainfall_flux / (mm/hour) (time: 1440; projection_y_coordinate: 519; projection_x_coordinate: 423)
    Dimension coordinates:
        time                              x                              -                             -
        projection_y_coordinate           -                              x                             -
        projection_x_coordinate           -                              -                             x
    Auxiliary coordinates:
        forecast_period                   x                              -                             -
    Scalar coordinates:
        forecast_reference_time      2059-09-01 00:30:00
        realization                  1100000
    Cell methods:
        mean                         time (1 hour)
    Attributes:
        C

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20789/848208593.py", line 67, in <module>
    iris.save(full_timeslice_cube, home_dir + f'datadir/Gauge_Timeslices/2002_2020/{em}/gauge{gauge_num}_farFuture.nc')
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/io/__init__.py", line 428, in save
    saver(source, target, **kwargs)
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/netcdf.py", line 2785, in save
    fill_value=fill_value,
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/netcdf.py", line 1165, in write
    fill_value=fill_value,
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/

TypeError: object of type 'NoneType' has no len()

In [8]:
import iris.coord_categorisation
print(gauge_num)
# Year isnt actually doing anything here 
cubes = load_files_to_cubelist(2061, general_filename)
print("Loaded data")

cubes = clean_cubes_v2(cubes)

# Assuming your CubeList is called `cube_list`
filtered_cubes = iris.cube.CubeList()

# Define the coordinates to filter by
y_idx = idx_2d[0]
x_idx = idx_2d[1]

# Loop through each cube in the CubeList and filter
for cube in cubes:
    iris.coord_categorisation.add_year(cube, 'time')
    # Extract the data at the specified y_idx and x_idx
    filtered_cube = cube[:, y_idx, x_idx]

    # Append the filtered cube to the new CubeList
    filtered_cubes.append(filtered_cube)

full_timeslice_cube = filtered_cubes.concatenate_cube()     

# Categorize the time coordinate to extract years
exclude_2080_constraint = iris.Constraint(year=lambda cell: cell != 2080)
full_timeslice_cube = full_timeslice_cube.extract(exclude_2080_constraint)


0
Loaded data


In [9]:
full_timeslice_cube

Stratiform Rainfall Flux (mm/hour),time
Shape,316800
Dimension coordinates,
time,x
Auxiliary coordinates,
year,x
Scalar coordinates,"projection_x_coordinate 436652.3976596222 m, bound=(435537.52118291694, 437765.57668372814) mprojection_y_coordinate 399280.9350159818 m, bound=(398161.5630884238, 400400.30992548843) m"
Cell methods,
mean time (1 hour),
Attributes,Conventions CF-1.7STASH m01s04i203source Data from Met Office Unified Modelum_version 10.6


In [6]:
full_timeslice_cube

Stratiform Rainfall Flux (mm/hour),time
Shape,326880
Dimension coordinates,
time,x
Auxiliary coordinates,
forecast_period,x
Scalar coordinates,
forecast_reference_time 2059-09-01 00,"0:00projection_x_coordinate 436652.3976596222 m, bound=(435537.52118291694, 437765.57668372814) mprojection_y_coordinate 399280.9350159818 m, bound=(398161.5630884238, 400400.30992548843) mrealization 1100000"
Cell methods,
mean time (1 hour),
Attributes,Conventions CF-1.7STASH m01s04i203source Data from Met Office Unified Modelum_version 10.6
